<a href="https://colab.research.google.com/github/esat-arslan/medicel_text_classificaiton/blob/main/ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
import os

In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/projeler/nlp

mtsamples.csv


In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/projeler/nlp/mtsamples.csv")
df.head()
df.shape



(4999, 6)

In [ ]:
df.columns

Index(['Unnamed: 0', 'description', 'medical_specialty', 'sample_name',
       'transcription', 'keywords'],
      dtype='object')

In [ ]:
!pip install simpletransformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.1/281.1 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 

In [ ]:
import pandas as pd
import os, sys
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
from os.path import join
from sklearn.metrics import f1_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import sklearn

import torch


In [ ]:
%pip install transformers
!pip install --upgrade simpletransformers==0.65.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.6/312.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.1/281.1 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB

In [ ]:
import pandas as pd
from sklearn.metrics import balanced_accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight
import torch

datapath = "data"

cuda_available = torch.cuda.is_available()

def eval_metrics(actual, pred):
    bal_acc = balanced_accuracy_score(actual, pred)
    f1_sc = f1_score(actual, pred, average="macro")
    return bal_acc, f1_sc

def data_loader(filename):
    df = pd.read_csv(filename)
    df.drop(['Unnamed: 0'], axis=1, inplace=True)

    counts = df['medical_specialty'].value_counts()
    others = [k for k, v in counts.items() if v < 100]
    for each_spec in others:
        df.loc[df['medical_specialty'] == each_spec, 'medical_specialty'] = 'others'

    counts = df['medical_specialty'].value_counts()
    print(counts)

    num_classes = len(df['medical_specialty'].unique())
    class_dict = dict(zip(df['medical_specialty'].unique(), list(range(num_classes))))
    df['medical_specialty'] = df['medical_specialty'].apply(lambda x: class_dict[x])
    df.dropna(inplace=True)
    df['transcription'] = df['keywords'] + df['transcription']
    X = df[['transcription']]
    y = df[["medical_specialty"]]

    train_x, test_x, train_y, test_y = train_test_split(X, y, stratify=y, test_size=0.25)
    #class_weights = sklearn.utils.class_weight.compute_class_weight("balanced",list(set(list(y.values))),list(y.values))
    #class_weights = sklearn.utils.class_weight.compute_class_weight("balanced", classes=np.unique(train_y.values.flatten()), y=train_y.values.flatten())
    class_weights = [1]*num_classes
    #class_weights = [1] * num_classes
    print(df.head())

    return train_x, train_y, test_x, test_y, num_classes, class_weights, class_dict

if __name__ == '__main__':
    train_x, train_y, test_x, test_y, num_classes, class_weights, class_dict = data_loader("/content/drive/MyDrive/projeler/nlp/mtsamples.csv")

    train_df = pd.DataFrame(columns=['text', 'labels'])
    train_df['text'] = train_x['transcription']
    train_df['labels'] = train_y['medical_specialty']
    print(train_df.head())
    test_df = pd.DataFrame(columns=['text', 'labels'])
    test_df['text'] = test_x['transcription']
    test_df['labels'] = test_y['medical_specialty']

    learning_rate = 1e-5
    num_of_epochs = 3


    model_args = ClassificationArgs(num_train_epochs=num_of_epochs, learning_rate=learning_rate, reprocess_input_data=True,
                                    save_model_every_epoch=False, overwrite_output_dir=True,)
    model = ClassificationModel(
        "roberta",
        "roberta-base",
        num_labels=num_classes,
        weight=class_weights,
        use_cuda=cuda_available,
        args=model_args
    )

    model.train_model(train_df)
    model.save_model()
    def custom_metrics(preds, labels):
        print(f"Shape of preds: {preds.shape}")
        print(f"Shape of labels: {labels.shape}")
        print(f"First 10 preds: {preds[:10]}")
        print(f"First 10 labels: {labels[:10]}")
        if len(preds.shape) > 1:
          preds = preds.argmax(axis=1)
        bal_acc = balanced_accuracy_score(labels, preds)
        f1 = f1_score(labels, preds, average='macro')
        return {
            "balanced_accuracy": bal_acc,
            "macro_f1": f1
        }
    result, model_outputs, wrong_predictions = model.eval_model(test_df,custom_metrics=custom_metrics)


    result, output = model.predict(test_df['text'].values.tolist())

    acc, f1 = eval_metrics(test_df['labels'], result)

    print(acc, f1)

medical_specialty
 Surgery                          1103
others                            1076
 Consult - History and Phy.        516
 Cardiovascular / Pulmonary        372
 Orthopedic                        355
 Radiology                         273
 General Medicine                  259
 Gastroenterology                  230
 Neurology                         223
 SOAP / Chart / Progress Notes     166
 Obstetrics / Gynecology           160
 Urology                           158
 Discharge Summary                 108
Name: count, dtype: int64
                                         description  medical_specialty  \
0   A 23-year-old white female presents with comp...                  0   
1           Consult for laparoscopic gastric bypass.                  0   
2           Consult for laparoscopic gastric bypass.                  0   
3                             2-D M-Mode. Doppler.                    1   
4                                 2-D Echocardiogram                  1   

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'clas

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/366 [00:00<?, ?it/s]

...

Shape of preds: (975,)
Shape of labels: (975,)
First 10 preds: [ 7.  5.  5.  6.  1.  5. 12.  2. 10.  0.]
First 10 labels: [ 7  5  5  6  1  5 12  2 10  0]


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

0.9788173882265134 0.9743299374784683


In [ ]:
test_df.to_csv("test_df.csv")

In [ ]:


saved_model = ClassificationModel(
    "roberta", "outputs"
)



In [ ]:
result,output = saved_model.predict(test_df['text'].values.tolist())
acc, f1 = eval_metrics(test_df['labels'],result)
print(acc,f1)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

0.9788173882265134 0.9743299374784683


In [ ]:
import pandas as pd
from sklearn.metrics import balanced_accuracy_score, f1_score, confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight
import torch

datapath = "data"

cuda_available = torch.cuda.is_available()

def eval_metrics(actual, pred):
    bal_acc = balanced_accuracy_score(actual, pred)
    f1_sc = f1_score(actual, pred, average="macro")
    return bal_acc, f1_sc

def data_loader(filename):
    df = pd.read_csv(filename)
    df.drop(['Unnamed: 0'], axis=1, inplace=True)

    counts = df['medical_specialty'].value_counts()
    others = [k for k, v in counts.items() if v < 100]
    for each_spec in others:
        df.loc[df['medical_specialty'] == each_spec, 'medical_specialty'] = 'others'

    counts = df['medical_specialty'].value_counts()
    print(counts)

    num_classes = len(df['medical_specialty'].unique())
    class_dict = dict(zip(df['medical_specialty'].unique(), list(range(num_classes))))
    df['medical_specialty'] = df['medical_specialty'].apply(lambda x: class_dict[x])
    df.dropna(inplace=True)
    df['transcription'] = df['keywords'] + df['transcription']
    X = df[['transcription']]
    y = df[["medical_specialty"]]

    train_x, test_x, train_y, test_y = train_test_split(X, y, stratify=y, test_size=0.25)
    #class_weights = sklearn.utils.class_weight.compute_class_weight("balanced",list(set(list(y.values))),list(y.values))
    #class_weights = sklearn.utils.class_weight.compute_class_weight("balanced", classes=np.unique(train_y.values.flatten()), y=train_y.values.flatten())
    class_weights = [1]*num_classes
    #class_weights = [1] * num_classes
    print(df.head())

    return train_x, train_y, test_x, test_y, num_classes, class_weights, class_dict

if __name__ == '__main__':
    train_x, train_y, test_x, test_y, num_classes, class_weights, class_dict = data_loader("/content/drive/MyDrive/projeler/nlp/mtsamples.csv")

    train_df = pd.DataFrame(columns=['text', 'labels'])
    train_df['text'] = train_x['transcription']
    train_df['labels'] = train_y['medical_specialty']
    print(train_df.head())
    test_df = pd.DataFrame(columns=['text', 'labels'])
    test_df['text'] = test_x['transcription']
    test_df['labels'] = test_y['medical_specialty']

    learning_rate = 1e-5
    num_of_epochs = 3


    model_args = ClassificationArgs(num_train_epochs=num_of_epochs, learning_rate=learning_rate, reprocess_input_data=True,
                                    save_model_every_epoch=False, overwrite_output_dir=True,)
    model = ClassificationModel(
        "distilbert",
        "medicalai/ClinicalBERT",
        tokenizer_name="medicalai/ClinicalBERT",
        num_labels=num_classes,
        weight=class_weights,
        use_cuda=cuda_available,
        args=model_args
    )

    model.train_model(train_df)
    model.save_model()
    def custom_metrics(preds, labels):
        print(f"Shape of preds: {preds.shape}")
        print(f"Shape of labels: {labels.shape}")
        print(f"First 10 preds: {preds[:10]}")
        print(f"First 10 labels: {labels[:10]}")
        if len(preds.shape) > 1:
          preds = preds.argmax(axis=1)
        bal_acc = balanced_accuracy_score(labels, preds)
        f1 = f1_score(labels, preds, average='macro')
        return {
            "balanced_accuracy": bal_acc,
            "macro_f1": f1
        }
    result, model_outputs, wrong_predictions = model.eval_model(test_df,custom_metrics=custom_metrics)


    result, output = model.predict(test_df['text'].values.tolist())

    acc, f1 = eval_metrics(test_df['labels'], result)

    print(acc, f1)
     # Confusion matrix
    conf_matrix = confusion_matrix(test_df['labels'], result)
    print("Confusion Matrix:\n", conf_matrix)

    # Classification report
    class_report = classification_report(test_df['labels'], result, target_names=class_dict.keys())
    print("Classification Report:\n", class_report)

    # Accuracy
    accuracy = accuracy_score(test_df['labels'], result)
    print("Accuracy: ", accuracy)

    # Print overall metrics
    print(f"Balanced Accuracy: {acc}, F1 Score: {f1}")

    # Optionally: visualize confusion matrix using seaborn or matplotlib
    import seaborn as sns
    import matplotlib.pyplot as plt

    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_dict.keys(), yticklabels=class_dict.keys())
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

medical_specialty
 Surgery                          1103
others                            1076
 Consult - History and Phy.        516
 Cardiovascular / Pulmonary        372
 Orthopedic                        355
 Radiology                         273
 General Medicine                  259
 Gastroenterology                  230
 Neurology                         223
 SOAP / Chart / Progress Notes     166
 Obstetrics / Gynecology           160
 Urology                           158
 Discharge Summary                 108
Name: count, dtype: int64
                                         description  medical_specialty  \
0   A 23-year-old white female presents with comp...                  0   
1           Consult for laparoscopic gastric bypass.                  0   
2           Consult for laparoscopic gastric bypass.                  0   
3                             2-D M-Mode. Doppler.                    1   
4                                 2-D Echocardiogram                  1   

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at medicalai/ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
np.quantile(transcription,0.95) #most used 100 keywords

894.0

In [ ]:
df_medical

,transcription,keywords
0,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."
...,...,...
4984,"ADMISSION DIAGNOSIS:, Morbid obesity. BMI is...","bariatrics, laparoscopic gastric bypass, gastr..."
4985,"HISTORY OF PRESENT ILLNESS:, Ms. A is a 55-ye...","bariatrics, jenny craig, medifast, nutrisystem..."
4989,"PAST MEDICAL HISTORY: ,She had a negative str...","bariatrics, elective surgical weight loss, sur..."
4993,"HISTORY:, A 55-year-old female presents self-...","allergy / immunology, chronic glossitis, xeros..."


In [ ]:
#%pip install accelerate bitsandbytes
from transformers import AutoTokenizer, AutoModel,AutoModelForSequenceClassification, DistilBertTokenizer
#unique_keywords = pd.concat([df_medical['keywords']]).unique()
#keyword_to_id = {keyword: idx for idx, keyword in enumerate(unique_keywords)}


In [ ]:
from transformers import AutoTokenizer, AutoModel
model_name = "medicalai/ClinicalBERT"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader



class MedicalKeywordDataset(Dataset):
    def __init__(self, df, transcript, keywords, tokenizer, in_len, out_len):
        self.df = df
        self.transcript = transcript
        self.keywords = keywords
        self.tokenizer = tokenizer
        self.in_len = in_len
        self.out_len = out_len


    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):

        transcript_tokens = self.tokenizer(self.df[self.transcript].iloc[idx],
                                           padding='max_length',
                                           truncation=True,
                                           max_length=self.in_len,
                                           return_tensors='pt'
                                          )['input_ids']
        keyword_tokens = self.tokenizer(self.df[self.keywords].iloc[idx],
                                       padding="max_length", truncation=True,
                                       max_length=self.out_len,
                                       return_tensors='pt')['input_ids']





        if torch.cuda.is_available():
            transcript_tokens = transcript_tokens.to("cuda")
            keyword_tokens = keyword_tokens.to("cuda")
        #print(transcript_tokens)
        return transcript_tokens[0,:], keyword_tokens[0,:]



In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df_medical, train_size=0.9, random_state=34)


ds_train = MedicalKeywordDataset(df_train,'transcription','keywords',tokenizer,
                            512, 100)

ds_test = MedicalKeywordDataset(df_test,'transcription','keywords',tokenizer,
                            512, 100)
batch_size = 8

dl_train = DataLoader(ds_train, batch_size=8, shuffle=True)
dl_test = DataLoader(ds_test, batch_size=8, shuffle=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [ ]:
import torch.nn as nn

In [ ]:
num_labels = 2

In [ ]:

### Optimizer Adam used here
optimizer = Adam(model.parameters(),lr=2e-4)

### Defining Epochs
epochs = 3

def num_batches(total, batch_size):
    if total % batch_size == 0:
        return total // batch_size
    else:
        return total // batch_size + 1


### Number of batches defined for a dataset
train_batches = num_batches(df_train.shape[0],batch_size)
test_batches = num_batches(df_test.shape[0],batch_size)

### Function to train model
def train(data,num_batches, model, optimizer):
    model.train()
    model_loss = 0
    model_acc = 0
    i = 0
    total_samples = 0
    for tr, kw in data:
        optimizer.zero_grad()
        print(kw.shape)
        print(tr.shape)
        #kw = kw[:, :tr.shape[1]] if kw.shape[1] >= tr.shape[1] else F.pad(kw, (0, tr.shape[1] - kw.shape[1]))

        ### Feed forward Pass
        out = model(tr)
        #out = model(input_ids=tr_input_ids, labels=kw_input_ids)
        ### Loss computation

        ### Accuracy Computation
        last_hidden_state = out.last_hidden_state.to(model.device)  # Get the last hidden state
        classifier = nn.Linear(last_hidden_state.shape[-1], num_labels).to(model.device)  # num_labels is the number of output labels
        logits = classifier(last_hidden_state)  # Get logits from classifier

        loss_fn = nn.CrossEntropyLoss()
        r_loss = loss_fn(out.logits.view(-1, out.logits.shape[-1]), kw.view(-1))
        model_loss += r_loss.item()

        ### Accuracy Computation
        preds = torch.softmax(logits,dim=2)
        preds = torch.argmax(preds,dim=2)
        acc = torch.sum(kw == preds).item()/(kw.shape[0]*kw.shape[1])
        model_acc += acc

        ### Backpropogation
        r_loss.backward()
        optimizer.step()

        i+=1
        print("[" + "="*(50*i//num_batches) + ">" +
              " "*(50*(1 - i//num_batches))
              + "]" + f"loss={model_loss/i} accuracy={model_acc/i}",
              end="\r")

    print("[" + "="*(50*i//num_batches) + ">" +
              " "*(50*(1 - i//num_batches))
              + "]" + f"loss={model_loss/i} accuracy={model_acc/i}",
              end="\n")


def test(data,num_batches, model):
    model.eval()
    model_loss = 0
    model_acc = 0
    i = 0
    for tr, kw in data:
        #optimzer.zero_grad()

        ### Feed forward Pass
        out = model(tr, labels=kw)

        ### Loss computation
        r_loss = out.loss
        model_loss += r_loss.item()

        ### Accuracy Computation
        logits = out.logits
        preds = torch.softmax(logits,dim=2)
        preds = torch.argmax(preds,dim=2)
        acc = torch.sum(kw == preds).item()/(kw.shape[0]*kw.shape[1])
        model_acc += acc

#         ### No Backpropogation as it is evaluation of model
#         r_loss.backward()
#         optimizer.step()

        i+=1
        print("[" + "="*(50*i//num_batches) + ">" +
              " "*(50*(1 - i//num_batches))
              + "]" + f"loss={model_loss/i} accuracy={model_acc/i}",
              end="\r")

    print("[" + "="*(50*i//num_batches) + ">" +
              " "*(50*(1 - i//num_batches))
              + "]" + f"loss={model_loss/i} accuracy={model_acc/i}",
              end="\n")

In [ ]:
  for e in range(epochs):
    train(dl_train,train_batches,model, optimizer)
    test(dl_test, test_batches,model)

torch.Size([8, 100])
torch.Size([8, 512])


AttributeError: 'BaseModelOutput' object has no attribute 'logits'

In [ ]:
def num_batches(total, batch_size):
    return (total + batch_size - 1) // batch_size

train_batches = num_batches(len(df_train), batch_size)
test_batches = num_batches(len(df_test), batch_size)


In [ ]:
def train(data, num_batches, model, optimizer):
    model.train()
    model_loss = 0
    model_acc = 0
    for i, (tr, kw) in enumerate(data):
        tr, kw = tr.to(device), kw.to(device)
        optimizer.zero_grad()

        outputs = model(input_ids=tr, labels=kw)

        loss = outputs.loss
        model_loss += loss.item()

        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        acc = (kw == preds).float().mean().item()
        model_acc += acc

        loss.backward()
        optimizer.step()

        progress = (i + 1) / num_batches
        print(f"[{'=' * int(progress * 50)}{' ' * (50 - int(progress * 50))}] "
              f"loss={model_loss / (i + 1):.4f} accuracy={model_acc / (i + 1):.4f}", end='\r')

    print(f"[{'=' * 50}] loss={model_loss / num_batches:.4f} accuracy={model_acc / num_batches:.4f}")

def test(data, num_batches, model):
    model.eval()
    model_loss = 0
    model_acc = 0
    with torch.no_grad():
        for i, (tr, kw) in enumerate(data):
            tr, kw = tr.to(device), kw.to(device)

            outputs = model(input_ids=tr, labels=kw)

            loss = outputs.loss
            model_loss += loss.item()

            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)
            acc = (kw == preds).float().mean().item()
            model_acc += acc

            progress = (i + 1) / num_batches
            print(f"[{'=' * int(progress * 50)}{' ' * (50 - int(progress * 50))}] "
                  f"loss={model_loss / (i + 1):.4f} accuracy={model_acc / (i + 1):.4f}", end='\r')

    print(f"[{'=' * 50}] loss={model_loss / num_batches:.4f} accuracy={model_acc / num_batches:.4f}")


In [ ]:
epochs = 3

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    print("Training...")
    train(dl_train, train_batches, model, optimizer)
    print("Testing...")
    test(dl_test, test_batches, model)
